# Bayes Tree Replay Memorization

In [1]:
import pandas as pd
import numpy as np
import sys
from sklearn.model_selection import train_test_split

import MySQLdb.cursors
import MySQLdb

import bayes_tree # contains bayes tree class
import config as cfg # contains unit lookups/helper functions/credentials

## Get Replay List and Prepare MySQL Cursors

In [2]:
database_list = ['sc_pvp','sc_pvt','tvt']  # 'sc_pvz' 'tvz','zvz'
connection_list = []
cursor_list = []
for database in database_list:
    db = MySQLdb.connect(
    host = cfg.credentials['host'],
    user = cfg.credentials['user'],
    passwd = cfg.credentials['passwd'],
    database = database,
    cursorclass = MySQLdb.cursors.SSCursor)
    connection_list.append(db)
    cursor_list.append(db.cursor())

In [3]:
!ls pickles

clustered_replays.pickle player_races.pickle      raw_train_builds.pickle
memo_tree.pickle         raw_id_list.pickle       test_replays.pickle
obs_id_list.pickle       raw_obs.pickle           unit_counts.pickle


In [4]:
clustered_replays = cfg.from_pickle('pickles/clustered_replays.pickle')
clustered_replays['playerreplayID'] = [item[1] for item in clustered_replays.index]
clustered_replays['cursor'] = [item[0] for item in clustered_replays.index]
clustered_replays.head()

,cluster,race,playerreplayID,cursor
"(sc_pvt, 5)",2,1,5,sc_pvt
"(sc_pvt, 8)",1,1,8,sc_pvt
"(sc_pvt, 10)",6,1,10,sc_pvt
"(sc_pvt, 11)",0,1,11,sc_pvt
"(sc_pvt, 14)",2,1,14,sc_pvt


In [5]:
train_replays, test_replays = train_test_split(clustered_replays, test_size=0.05, random_state=42)
cfg.to_pickle(test_replays,'test_replays')

## Generate Build Trees

In [6]:
def get_builds(playerreplayIDs,time,cursor):
    n = 300
    results = []
    request_chunks = [playerreplayIDs[i * n:(i + 1) * n] for i in range((len(playerreplayIDs) + n - 1) // n )]
    for segment in request_chunks:
        playerreplayIDs_string = ",".join(str(x) for x in segment)
        query = """
            SELECT playerreplay.PlayerReplayID, unit.UnitTypeID, CAST(ChangeTime / 24 AS UNSIGNED) AS time 
            FROM (attributechange LEFT JOIN unit ON unit.UnitID = attributechange.UnitID) 
            LEFT JOIN playerreplay ON playerreplay.PlayerReplayID = unit.PlayerReplayID
            WHERE AttributeTypeID = 28 AND unit.PlayerReplayID in ({}) AND ChangeVal = 1 AND ChangeTime <= {}
            ORDER BY ChangeTime,UnitTypeID;
        """.format(playerreplayIDs_string,time)
        print(query)
    
        cursor.execute(query)
        results += cursor.fetchall()
        
    print('done!')
    ids = set(map(lambda x:x[0], results))
    results_by_ids = [[y[1:3] for y in results if y[0]==x] for x in ids]
    
    return results_by_ids,ids


def process_builds(results):
    
    build_list = []
    
    for result in results:
    
        build_order = tuple([x[0] for x in result])
        
        for index,item in enumerate(build_order):
            if item in cfg.tree_converts.keys():
                build_order[index] = cfg.tree_converts[item]

        build_tup = []
        for last_elem in range(1,len(build_order)+1):
            seq = build_order[0:last_elem]
            if seq[-1] not in cfg.tree_choices.keys():
                continue
            build_tup.append((seq[-1],seq.count(seq[-1])))
        timing = tuple([x[1] for x in result])
        build_list.append({'build' : build_tup , 'time' : timing})
    return build_list

In [7]:
try:
    raw_builds = cfg.from_pickle('pickles/raw_train_builds.pickle')
    id_list = cfg.from_pickle('pickles/raw_id_list.pickle')
except:
    print('generating builds')
    raw_builds = []
    id_list = []
    for index,database in enumerate(database_list):
        playerreplayIDs = train_replays['playerreplayID'][train_replays['cursor'] == database]
        cursor = cursor_list[index]
        builds, ids = get_builds(playerreplayIDs,cfg.TIME,cursor)
        raw_builds.append(builds)
        id_list.append(ids)
    
    cfg.to_pickle(raw_builds,'raw_train_builds')
    cfg.to_pickle(id_list,'raw_id_list')

In [8]:
processed_builds = []
for builds in raw_builds:
    processed_builds.append(process_builds(builds))

In [9]:
all_builds = []
all_ids = []
all_unique_ids = []
all_races = []
for matchup_ind,matchup in enumerate(processed_builds):
    for match_ind,match in enumerate(matchup):
        
        live_id = list(id_list[matchup_ind])[match_ind]
        all_ids.append(live_id)
        unique_id = (database_list[matchup_ind],live_id)
        all_unique_ids.append(unique_id)
        
        race = clustered_replays['race'][clustered_replays.index == unique_id].values[0]
        all_races.append(race)
     
        match['cluster'] = clustered_replays['cluster'][clustered_replays.index == unique_id].values[0]
        match['freq'] = 1 / len(train_replays)
        match['name'] = [f'{x[1]}x {cfg.tree_choices[x[0]]}' for x in match['build']]
           
        all_builds.append(match)

In [11]:
tree = bayes_tree.tree()
counter = 0
for build in all_builds:
    counter += 1
    print(f'Adding build #{counter}')
    tree.add_build(build)
tree.fit_nodes(3,15) #3 obs cutoff, 15 second default std

Adding build #1
Adding build #2
Adding build #3
Adding build #4
Adding build #5
Adding build #6
Adding build #7
Adding build #8
Adding build #9
Adding build #10
Adding build #11
Adding build #12
Adding build #13
Adding build #14
Adding build #15
Adding build #16
Adding build #17
Adding build #18
Adding build #19
Adding build #20
Adding build #21
Adding build #22
Adding build #23
Adding build #24
Adding build #25
Adding build #26
Adding build #27
Adding build #28
Adding build #29
Adding build #30
Adding build #31
Adding build #32
Adding build #33
Adding build #34
Adding build #35
Adding build #36
Adding build #37
Adding build #38
Adding build #39
Adding build #40
Adding build #41
Adding build #42
Adding build #43
Adding build #44
Adding build #45
Adding build #46
Adding build #47
Adding build #48
Adding build #49
Adding build #50
Adding build #51
Adding build #52
Adding build #53
Adding build #54
Adding build #55
Adding build #56
Adding build #57
Adding build #58
Adding build #59
Adding

Adding build #552
Adding build #553
Adding build #554
Adding build #555
Adding build #556
Adding build #557
Adding build #558
Adding build #559
Adding build #560
Adding build #561
Adding build #562
Adding build #563
Adding build #564
Adding build #565
Adding build #566
Adding build #567
Adding build #568
Adding build #569
Adding build #570
Adding build #571
Adding build #572
Adding build #573
Adding build #574
Adding build #575
Adding build #576
Adding build #577
Adding build #578
Adding build #579
Adding build #580
Adding build #581
Adding build #582
Adding build #583
Adding build #584
Adding build #585
Adding build #586
Adding build #587
Adding build #588
Adding build #589
Adding build #590
Adding build #591
Adding build #592
Adding build #593
Adding build #594
Adding build #595
Adding build #596
Adding build #597
Adding build #598
Adding build #599
Adding build #600
Adding build #601
Adding build #602
Adding build #603
Adding build #604
Adding build #605
Adding build #606
Adding bui

Adding build #1086
Adding build #1087
Adding build #1088
Adding build #1089
Adding build #1090
Adding build #1091
Adding build #1092
Adding build #1093
Adding build #1094
Adding build #1095
Adding build #1096
Adding build #1097
Adding build #1098
Adding build #1099
Adding build #1100
Adding build #1101
Adding build #1102
Adding build #1103
Adding build #1104
Adding build #1105
Adding build #1106
Adding build #1107
Adding build #1108
Adding build #1109
Adding build #1110
Adding build #1111
Adding build #1112
Adding build #1113
Adding build #1114
Adding build #1115
Adding build #1116
Adding build #1117
Adding build #1118
Adding build #1119
Adding build #1120
Adding build #1121
Adding build #1122
Adding build #1123
Adding build #1124
Adding build #1125
Adding build #1126
Adding build #1127
Adding build #1128
Adding build #1129
Adding build #1130
Adding build #1131
Adding build #1132
Adding build #1133
Adding build #1134
Adding build #1135
Adding build #1136
Adding build #1137
Adding build

Adding build #1566
Adding build #1567
Adding build #1568
Adding build #1569
Adding build #1570
Adding build #1571
Adding build #1572
Adding build #1573
Adding build #1574
Adding build #1575
Adding build #1576
Adding build #1577
Adding build #1578
Adding build #1579
Adding build #1580
Adding build #1581
Adding build #1582
Adding build #1583
Adding build #1584
Adding build #1585
Adding build #1586
Adding build #1587
Adding build #1588
Adding build #1589
Adding build #1590
Adding build #1591
Adding build #1592
Adding build #1593
Adding build #1594
Adding build #1595
Adding build #1596
Adding build #1597
Adding build #1598
Adding build #1599
Adding build #1600
Adding build #1601
Adding build #1602
Adding build #1603
Adding build #1604
Adding build #1605
Adding build #1606
Adding build #1607
Adding build #1608
Adding build #1609
Adding build #1610
Adding build #1611
Adding build #1612
Adding build #1613
Adding build #1614
Adding build #1615
Adding build #1616
Adding build #1617
Adding build

Adding build #2000
Adding build #2001
Adding build #2002
Adding build #2003
Adding build #2004
Adding build #2005
Adding build #2006
Adding build #2007
Adding build #2008
Adding build #2009
Adding build #2010
Adding build #2011
Adding build #2012
Adding build #2013
Adding build #2014
Adding build #2015
Adding build #2016
Adding build #2017
Adding build #2018
Adding build #2019
Adding build #2020
Adding build #2021
Adding build #2022
Adding build #2023
Adding build #2024
Adding build #2025
Adding build #2026
Adding build #2027
Adding build #2028
Adding build #2029
Adding build #2030
Adding build #2031
Adding build #2032
Adding build #2033
Adding build #2034
Adding build #2035
Adding build #2036
Adding build #2037
Adding build #2038
Adding build #2039
Adding build #2040
Adding build #2041
Adding build #2042
Adding build #2043
Adding build #2044
Adding build #2045
Adding build #2046
Adding build #2047
Adding build #2048
Adding build #2049
Adding build #2050
Adding build #2051
Adding build

...

Adding build #2983
Adding build #2984
Adding build #2985
Adding build #2986
Adding build #2987
Adding build #2988
Adding build #2989
Adding build #2990
Adding build #2991
Adding build #2992
Adding build #2993
Adding build #2994
Adding build #2995
Adding build #2996
Adding build #2997
Adding build #2998
Adding build #2999
Adding build #3000
Adding build #3001
Adding build #3002
Adding build #3003
Adding build #3004
Adding build #3005
Adding build #3006
Adding build #3007
Adding build #3008
Adding build #3009
Adding build #3010
Adding build #3011
Adding build #3012
Adding build #3013
Adding build #3014
Adding build #3015
Adding build #3016
Adding build #3017
Adding build #3018
Adding build #3019
Adding build #3020
Adding build #3021
Adding build #3022
Adding build #3023
Adding build #3024
Adding build #3025
Adding build #3026
Adding build #3027
Adding build #3028
Adding build #3029
Adding build #3030
Adding build #3031
Adding build #3032
Adding build #3033
Adding build #3034
Adding build

Adding build #3473
Adding build #3474
Adding build #3475
Adding build #3476
Adding build #3477
Adding build #3478
Adding build #3479
Adding build #3480
Adding build #3481
Adding build #3482
Adding build #3483
Adding build #3484
Adding build #3485
Adding build #3486
Adding build #3487
Adding build #3488
Adding build #3489
Adding build #3490
Adding build #3491
Adding build #3492
Adding build #3493
Adding build #3494
Adding build #3495
Adding build #3496
Adding build #3497
Adding build #3498
Adding build #3499
Adding build #3500
Adding build #3501
Adding build #3502
Adding build #3503
Adding build #3504
Adding build #3505
Adding build #3506
Adding build #3507
Adding build #3508
Adding build #3509
Adding build #3510
Adding build #3511
Adding build #3512
Adding build #3513
Adding build #3514
Adding build #3515
Adding build #3516
Adding build #3517
Adding build #3518
Adding build #3519
Adding build #3520
Adding build #3521
Adding build #3522
Adding build #3523
Adding build #3524
Adding build

Adding build #3947
Adding build #3948
Adding build #3949
Adding build #3950
Adding build #3951
Adding build #3952
Adding build #3953
Adding build #3954
Adding build #3955
Adding build #3956
Adding build #3957
Adding build #3958
Adding build #3959
Adding build #3960
Adding build #3961
Adding build #3962
Adding build #3963
Adding build #3964
Adding build #3965
Adding build #3966
Adding build #3967
Adding build #3968
Adding build #3969
Adding build #3970
Adding build #3971
Adding build #3972
Adding build #3973
Adding build #3974
Adding build #3975
Adding build #3976
Adding build #3977
Adding build #3978
Adding build #3979
Adding build #3980
Adding build #3981
Adding build #3982
Adding build #3983
Adding build #3984
Adding build #3985
Adding build #3986
Adding build #3987
Adding build #3988
Adding build #3989
Adding build #3990
Adding build #3991
Adding build #3992
Adding build #3993
Adding build #3994
Adding build #3995
Adding build #3996
Adding build #3997
Adding build #3998
Adding build

Adding build #4481
Adding build #4482
Adding build #4483
Adding build #4484
Adding build #4485
Adding build #4486
Adding build #4487
Adding build #4488
Adding build #4489
Adding build #4490
Adding build #4491
Adding build #4492
Adding build #4493
Adding build #4494
Adding build #4495
Adding build #4496
Adding build #4497
Adding build #4498
Adding build #4499
Adding build #4500
Adding build #4501
Adding build #4502
Adding build #4503
Adding build #4504
Adding build #4505
Adding build #4506
Adding build #4507
Adding build #4508
Adding build #4509
Adding build #4510
Adding build #4511
Adding build #4512
Adding build #4513
Adding build #4514
Adding build #4515
Adding build #4516
Adding build #4517
Adding build #4518
Adding build #4519
Adding build #4520
Adding build #4521
Adding build #4522
Adding build #4523
Adding build #4524
Adding build #4525
Adding build #4526
Adding build #4527
Adding build #4528
Adding build #4529
Adding build #4530
Adding build #4531
Adding build #4532
Adding build

Adding build #4977
Adding build #4978
Adding build #4979
Adding build #4980
Adding build #4981
Adding build #4982
Adding build #4983
Adding build #4984
Adding build #4985
Adding build #4986
Adding build #4987
Adding build #4988
Adding build #4989
Adding build #4990
Adding build #4991
Adding build #4992
Adding build #4993
Adding build #4994
Adding build #4995
Adding build #4996
Adding build #4997
Adding build #4998
Adding build #4999
Adding build #5000
Adding build #5001
Adding build #5002
Adding build #5003
Adding build #5004
Adding build #5005
Adding build #5006
Adding build #5007
Adding build #5008
Adding build #5009
Adding build #5010
Adding build #5011
Adding build #5012
Adding build #5013
Adding build #5014
Adding build #5015
Adding build #5016
Adding build #5017
Adding build #5018
Adding build #5019
Adding build #5020
Adding build #5021
Adding build #5022
Adding build #5023
Adding build #5024
Adding build #5025
Adding build #5026
Adding build #5027
Adding build #5028
Adding build

In [12]:
sys.setrecursionlimit(100000)
cfg.to_pickle(tree,'memo_tree')
sys.setrecursionlimit(1000)